#### Reactions processing with AQME - substrates + TS


In [ ]:
# cell with import, system name and PATHs
import os, glob, subprocess
import shutil
from pathlib import Path
from aqme.csearch import csearch
from aqme.qprep import qprep
from aqme.qcorr import qcorr
from rdkit import Chem
import pandas as pd

###### Step 1: Determining the constraints for SN2 TS

In [ ]:
# Provide the TS smiles to detemine the numbering for constraints
smi = 'C(C)(F)C.[OH-]'
mol = Chem.MolFromSmiles(smi)
mol = Chem.AddHs(mol)
for i,atom in enumerate(mol.GetAtoms()):
    atom.SetAtomMapNum(i)
smi_new = Chem.MolToSmiles(mol)
print(smi_new)

In [ ]:
mol
# distance and angle to fix are 
# constraits_dist = [[0,2,1.8],[0,4,1.8]]
# constraits_angle = [[2,0,4,180]]

###### Step 2: Create a CSV as follows

In [ ]:
data = pd.read_csv('example2.csv')
data

###### Step 3: Running CSEARCH on the CSV

In [ ]:
# run CSEARCH conformational sampling, specifying:

# choose program for conformer sampling
# 1) RDKit ('rdkit'): Fast sampling, only works for systems with one molecule
# 2) CREST ('crest'): Slower sampling, works for noncovalent complexes and 
# transition structures (see example of TS in the CSEARCH_CREST_TS.ipynb notebook
#  from the CSEARCH_CMIN_conformer_generation folder)

# 3) Program for conformer sampling (program=program)
# 4) SMILES string (smi=smi)
# 5) Name for the output SDF files (name=name)
# 6) Include CREGEN post-analysis for CREST sampling (cregen=True)
csearch(input='example2.csv',  program='crest', cregen=True, cregen_keywords='--ethr 0.1 --rthr 0.2 --bthr 0.3 --ewin 1')

###### Step 4: Create input files using QPREP 

###### a. for TS with TS keywords
###### b. for substrates with substrate keywords

In [ ]:
# set SDF filenames and directory where the new com files will be created
sdf_rdkit_files = ['CSEARCH/crest/TS_SN2_crest.sdf']

# choose program for input file generation, with the corresponding keywords line, memory and processors:
# 1) Gaussian ('gaussian')
program = 'gaussian'
qm_input = 'B3LYP/6-31G(d) opt=(ts,calcfc,noeigen) freq'
mem='40GB'
nprocs=36

# run QPREP input files generator, with:
# 1) Working directory (w_dir_main=sdf_path)
# 2) PATH to create the new SDF files (destination=com_path)
# 3) Files to convert (files=sdf_rdkit_files)
# 4) QM program for the input (program=program)
# 5) Keyword line for the Gaussian inputs (qm_input=qm_input)
# 6) Memory to use in the calculations (mem='24GB')
# 7) Processors to use in the calcs (nprocs=8)
qprep(files=sdf_rdkit_files,program=program,
        qm_input=qm_input,mem=mem,nprocs=nprocs)
 

In [ ]:
# set SDF filenames and directory where the new com files will be created
sdf_rdkit_files = ['CSEARCH/crest/F_crest.sdf', 'CSEARCH/crest/O_anion_crest.sdf']

# choose program for input file generation, with the corresponding keywords line, memory and processors:
# 1) Gaussian ('gaussian')
program = 'gaussian'
qm_input = 'B3LYP/6-31G(d) opt freq'
mem='40GB'
nprocs=36

# run QPREP input files generator, with:
# 1) Working directory (w_dir_main=sdf_path)
# 2) PATH to create the new SDF files (destination=com_path)
# 3) Files to convert (files=sdf_rdkit_files)
# 4) QM program for the input (program=program)
# 5) Keyword line for the Gaussian inputs (qm_input=qm_input)
# 6) Memory to use in the calculations (mem='24GB')
# 7) Processors to use in the calcs (nprocs=8)
qprep(files=sdf_rdkit_files,program=program,
        qm_input=qm_input,mem=mem,nprocs=nprocs)

###### Step 5: Checking with QPREP for corrections

In [ ]:
w_dir_main=os.getcwd()+'/QCALC'

# run the QCORR analyzer, with:
# 1) Working directory (w_dir_main=com_path)
# 2) Names of the QM output files (files='*.log')
# 3) Detect and fix calcs that converged during geometry optimization but didn't converge during frequency calcs (freq_conv='opt=(calcfc,maxstep=5)')
# 4) Type of initial input files where the LOG files come from (isom_type='com')
# 5) Folder with the initial input files (isom_inputs=com_path)

qcorr(w_dir_main=w_dir_main,files='*.log',freq_conv='opt=(calcfc,maxstep=5)')

###### Step 6: creation of DLPNO input files for ORCA single-point energy calculations

In [ ]:
# choose output files to get atoms and coordinates to generate inputs for single-point energy calculations
success_dir = os.getcwd()+'/QCALC/successful_QM_outputs'
qm_files = '*.log'

# choose program for input file generation with QPREP, with the corresponding keywords line, memory and processors:

# 1) ORCA ('orca')
program = 'orca'
# a DLPNO example keywords line for ORCA calculations
# qm_input = 'Extrapolate(2/3,cc) def2/J cc-pVTZ/C DLPNO-CCSD(T) NormalPNO TightSCF RIJCOSX\n'
qm_input ='DLPNO-CCSD(T) def2-tzvpp def2-tzvpp/C\n'
qm_input += '%scf maxiter 500\n'
qm_input += 'end\n'
qm_input += '% mdci\n'
qm_input += 'Density None\n'
qm_input += 'end\n'
qm_input += '% elprop\n'
qm_input += 'Dipole False\n'
qm_input += 'end'
mem='4GB'
nprocs=8

# run QPREP input files generator, with:
# 1) Working directory (w_dir_main=sdf_path)
# 2) PATH to create the new SDF files (destination=com_path)
# 3) Files to convert (files=sdf_rdkit_files)
# 4) QM program for the input (program=program)
# 5) Keyword line for the Gaussian inputs (qm_input=qm_input)
# 6) Memory to use in the calculations (mem='24GB')
# 7) Processors to use in the calcs (nprocs=8)

qprep(w_dir_main=success_dir,destination=success_dir,files=qm_files,program=program,
        qm_input=qm_input,mem=mem,nprocs=nprocs, suffix='DLPNO')

###### Step 7: Analysis with goodvibes

In [ ]:
# track all the output files from Gaussian and ORCA
opt_files = glob.glob(f'{success_dir}/*.log')
spc_files = glob.glob(f'{success_dir}/*.out')
all_files = opt_files + spc_files

# move all the output files together to a folder called "GoodVibes_analysis" for simplicity
w_dir_main  = Path(os.getcwd())
GV_folder = w_dir_main.joinpath('GoodVibes_analysis')
GV_folder.mkdir(exist_ok=True, parents=True)

for file in all_files:
	shutil.copy(file, GV_folder)

# this commands runs GoodVibes, including the population % of each conformer 
# (final results in the GoodVibes.out file)
os.chdir(GV_folder)
subprocess.run(['python', '-m', 'goodvibes', '--xyz','--pes', '../pes.yaml','--graph','../pes.yaml', '--spc', 'DLPNO', '*.log',])
os.chdir(w_dir_main)